In [1]:
#Import libraries

import pandas as pd
import numpy as np
import time
import datetime

from twython import Twython
from email.utils import mktime_tz, parsedate_tz

In [4]:
#Define functions

def ambil_hashtag(dicts):
    hashtag = []
    for i in range(len(dicts)):
        hashtag.append(dicts[i]['text'])
    return hashtag

def ambil_mention(dicts):
    mention = []
    for i in range(len(dicts)):
        mention.append(dicts[i]['screen_name'])
    return mention

def parse_datetime(value):
    time_tuple = parsedate_tz(value)
    timestamp = mktime_tz(time_tuple)

    return datetime.datetime.fromtimestamp(timestamp)

In [2]:
#Input Consumer API keys from Twitter

APP_KEY = 'AAAA'
APP_SECRET = 'BBBB'

#Get OAuth 2 Access Token from Consumer API keys
twitter = Twython(APP_KEY, APP_SECRET, oauth_version=2)
ACCESS_TOKEN = twitter.obtain_access_token()

#Create a Twython instance with your application key and access token
API = Twython(APP_KEY, access_token=ACCESS_TOKEN)

In [3]:
#Collect data from Twitter

tweets = []
MAX_ATTEMPTS = 100
COUNT_OF_TWEETS_TO_BE_FETCHED = 10000
que='CidukSaidDidu'

#For each attempt, this code get 100 tweets from Twitter search. 
#If you want to get 10.000 tweets, you should set MAX_ATTEMPTS to 100 and so on.

for i in range(0,MAX_ATTEMPTS):

    if(COUNT_OF_TWEETS_TO_BE_FETCHED < len(tweets)):
        break

    #----------------------------------------------------------------#
    # STEP 1: Query Twitter
    # STEP 2: Save the returned tweets
    # STEP 3: Get the next max_id
    #----------------------------------------------------------------#

    # STEP 1: Query Twitter
    if(0 == i):
        # Query twitter for data. 
        results = API.search(q=que, include_entities='true', count='100', tweet_mode='extended')
    else:
        # After the first call we should have max_id from result of previous call. Pass it in query.
        results = API.search(q=que, include_entities='true', count='100', tweet_mode='extended', max_id=next_max_id)

    # STEP 2: Save the returned tweets
    for result in results['statuses']:
        tweets.append(result)

    # STEP 3: Get the next max_id
    try:
        # Parse the data returned to get max_id to be passed in consequent call.
        next_results_url_params = results['search_metadata']['next_results']
        next_max_id = next_results_url_params.split('max_id=')[1].split('&')[0]
    except:
        # No more next pages
        break

In [5]:
df = pd.DataFrame(tweets)
df.head()

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,full_text,geo,...,possibly_sensitive,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,truncated,user
0,None,None,Tue May 05 16:58:49 +0000 2020,"[0, 139]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,RT @HusinShihab: Bagus nih penjelasannya bib @...,None,...,NaN,NaN,NaN,NaN,467,False,{'created_at': 'Mon May 04 20:24:08 +0000 2020...,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 1142488713655545856, 'id_str': '1142488..."
1,None,None,Tue May 05 16:49:40 +0000 2020,"[0, 139]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,RT @HusinShihab: Bagus nih penjelasannya bib @...,None,...,NaN,NaN,NaN,NaN,467,False,{'created_at': 'Mon May 04 20:24:08 +0000 2020...,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 3327772232, 'id_str': '3327772232', 'na..."
2,None,None,Tue May 05 16:36:46 +0000 2020,"[0, 139]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,RT @HusinShihab: Bagus nih penjelasannya bib @...,None,...,NaN,NaN,NaN,NaN,467,False,{'created_at': 'Mon May 04 20:24:08 +0000 2020...,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 1032296027779301376, 'id_str': '1032296..."
3,None,None,Tue May 05 16:08:15 +0000 2020,"[0, 139]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,RT @HusinShihab: Bagus nih penjelasannya bib @...,None,...,NaN,NaN,NaN,NaN,467,False,{'created_at': 'Mon May 04 20:24:08 +0000 2020...,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 344520273, 'id_str': '344520273', 'name..."
4,None,None,Tue May 05 16:06:54 +0000 2020,"[0, 139]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,RT @HusinShihab: Bagus nih penjelasannya bib @...,None,...,NaN,NaN,NaN,NaN,467,False,{'created_at': 'Mon May 04 20:24:08 +0000 2020...,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 1075694112165294080, 'id_str': '1075694..."


In [6]:
#Get important variable

ID_TWEETS = df['id']

ENTITIES = df['entities'].apply(pd.Series)
HASHTAG = ENTITIES['hashtags'].apply(ambil_hashtag)
MENTION = ENTITIES['user_mentions'].apply(ambil_mention)

TIME = df['created_at']
TEXT = df['full_text']

USER = df['user'].apply(pd.Series)
USER = USER.rename(columns={'created_at':'user_time'})

In [7]:
data = pd.concat([ID_TWEETS, TIME, USER['screen_name'], USER['user_time'], USER['followers_count'], USER['friends_count'], TEXT, MENTION, HASHTAG], axis=1)
data = data.rename(columns={'id':'id_tweets','created_at':'time', 'screen_name':'user', 'user_time':'joined', 'followers_count':'followers', 'friends_count':'following', 'full_text':'text', 'user_mentions':'mentions', 'hashtags':'hashtags'})
data.head()

,id_tweets,time,user,joined,followers,following,text,mentions,hashtags
0,1257716392331104259,Tue May 05 16:58:49 +0000 2020,agustin_sastra,Sat Jun 22 17:45:10 +0000 2019,616,405,RT @HusinShihab: Bagus nih penjelasannya bib @...,"[HusinShihab, muannas_alaidid, msaid_didu]",[]
1,1257714088131432449,Tue May 05 16:49:40 +0000 2020,eddy_AS_111,Mon Aug 24 03:27:10 +0000 2015,204,1172,RT @HusinShihab: Bagus nih penjelasannya bib @...,"[HusinShihab, muannas_alaidid, msaid_didu]",[]
2,1257710843325644800,Tue May 05 16:36:46 +0000 2020,CintaDamai_10,Wed Aug 22 15:58:46 +0000 2018,28,128,RT @HusinShihab: Bagus nih penjelasannya bib @...,"[HusinShihab, muannas_alaidid, msaid_didu]",[]
3,1257703665890025474,Tue May 05 16:08:15 +0000 2020,ibrocksfuck38,Fri Jul 29 05:22:41 +0000 2011,308,502,RT @HusinShihab: Bagus nih penjelasannya bib @...,"[HusinShihab, muannas_alaidid, msaid_didu]",[]
4,1257703323626377216,Tue May 05 16:06:54 +0000 2020,NataprajaSule,Thu Dec 20 10:07:16 +0000 2018,435,412,RT @HusinShihab: Bagus nih penjelasannya bib @...,"[HusinShihab, muannas_alaidid, msaid_didu]",[]


In [8]:
#Change time zone to WIB (GMT +7)

tweets_time=[]
for row in data.itertuples():
    ts = parse_datetime(row.time)
    tweets_time.append(str(ts))
    
data['time'] = tweets_time

joined_time=[]
for row in data.itertuples():
    ts = parse_datetime(row.joined)
    joined_time.append(str(ts))
    
data['joined'] = joined_time

In [9]:
#Save data frame to excel

data.to_excel(r'..\giveaway_CidukSaidDidu.xlsx', index=False)